In [ ]:
from IPython.utils import capture
from IPython.display import clear_output
import os

#@markdown **WebUI Version 설치한 버전에 맞게 선택해주세요**
TagVersion = 'v1.7.0' #@param ["v1.3.2", "v1.4.1", "v1.5.2", "v1.6.0", "v1.7.0", "v1.8.0", "v1.9.0", "v1.10.0"]

#@markdown **download_package (미리구성된 패키지) / pip_install_torch (일반설치)**
#@markdown > *실행시 모듈 오류가 날때는 pip_install_torch 를 선택해주세요*  
RuntimeEnv = 'pip_install_torch' #@param ["download_package", "pip_install_torch"]

#@markdown **체크포인트(checkpoint) 다운로드 URL**
Checkpoint_Url = '' #@param {type:"string"}
#@markdown **체크포인트 저장 파일이름**
Checkpoint_Filename = 'checkpoint.safetensors' #@param {type:"string"}

#@markdown **ControlNet 기본 모델 다운로드**
#@markdown > *openpose, lineart, softedge, depth, canny, tile, ip2p*  
ControlNet = True #@param {type:"boolean"}

#@markdown **ControlNet 추가 모델 다운로드**
#@markdown > *segment, inpaint, scribble, mlsd, normalbae, suffle*  
ControlNet_Extra = False #@param {type:"boolean"}

#@markdown **IP-Adapter 모델 다운로드**
CotnrolNet_IPAdapter = False #@param {type:"boolean"}

#@markdown **TemporalNet 모델 다운로드**
#@markdown > *temporalnet*  
ControlNet_TemporalNet = False #@param {type:"boolean"}

#@markdown **T2I Adapter 모델 다운로드**
#@markdown > *style, color, sketch, zoedepth, recolor*  
ControlNet_T2I = False #@param {type:"boolean"}

#@markdown **AnimateDiff 모델 다운로드**
AnimateDiff = False #@param {type:"boolean"}

#@markdown **AnimateDiff 추가 모션 모델 (HumansMotion,3DMotion)**
AnimateDiff_Extra = False #@param {type:"boolean"}

#@markdown **LCM-LoRa 사용**
LCMLora = False #@param {type:"boolean"}

#@markdown **Temporal Kit 사용**
TemporalKit = False #@param {type:"boolean"}

#@markdown **TemporalNet Util 사용**
TemporalNetUtil = False #@param {type:"boolean"}

Workspace = 'install'
ControlNet_Model_Path = "/content/controlnet_models"
AnimateDiff_Model_Path = "/content/motion_models"

from google.colab import drive
drive.mount('/content/drive')

print('install...')
with capture.capture_output() as cap:
  !apt -y install -qq aria2
  if RuntimeEnv == 'download_package':
    exclude_list = [
        "usr/local/lib/python3.10/dist-packages/Pillow*",
        "usr/local/lib/python3.10/dist-packages/PIL*",
        "usr/local/lib/python3.10/dist-packages/numpy*",
        "usr/local/lib/python3.10/dist-packages/regex*",
    ]
    with open('exclude.txt', 'w') as f:
        for item in exclude_list:
            f.write(f"{item}\n")

    !pip install trash-cli
    !trash-put /usr/local/lib/python3.10/dist-packages/torch
    !apt install zstd
    !wget https://huggingface.co/ninjaneural/webui/resolve/main/direct/lib_20240614.zst
    !tar -C / --zstd -xvf lib_20240614.zst -X exclude.txt
    !rm lib_20240614.zst
    !trash-put /usr/local/lib/python3.10/dist-packages/rpds_py*
    !pip install rembg
    !trash-put /usr/local/lib/python3.10/dist-packages/notebook_shim*
    !trash-put /usr/local/lib/python3.10/dist-packages/platformdirs*
    !pip install gfpgan 
    !pip install realesrgan
    !pip install modelscope
    !pip install ffmpeg-python
    if TagVersion=='v1.9.0':
      !trash-put /usr/local/lib/python3.10/dist-packages/filelock* 
    if TagVersion=='v1.10.0':
      !trash-put /usr/local/lib/python3.10/dist-packages/filelock* 
  else:
    !wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/install_torch.sh -O /content/install.sh
    !bash /content/install.sh
    !pip install pytorch_lightning==1.9.4
    !pip install gradio==3.41.2
    !sed -i 's/\[block\._id for block in inputs\]/[block._id for block in inputs if block is not None]/g; s/\[block\._id for block in outputs\]/[block._id for block in outputs if block is not None]/g' /usr/local/lib/python3.10/dist-packages/gradio/blocks.py
    !pip install pydantic==1.10.17
    !pip install gfpgan
    !pip install omegaconf
    !pip install kornia
    !pip install open_clip_torch
    !pip install piexif
    !pip install jsonmerge
    !pip install torchdiffeq
    !pip install torchsde
    !pip install clean-fid
    !pip install clip
    !pip install resize_right
    !pip install lark
    !pip install tomesd
    !pip install blendmodes
    !pip install inflection
    !pip install facexlib

  !pip install diskcache
  !pip install segment_anything
  !pip install insightface
  if TagVersion=='v1.9.0':
    !pip install dynamicprompts[attentiongrabber,magicprompt]~=0.31.0
    !pip install controlnet_aux
  if TagVersion=='v1.10.0':
    !pip install dynamicprompts[attentiongrabber,magicprompt]~=0.31.0
    !pip install controlnet_aux
    !pip install pillow-avif-plugin


!mkdir -p {ControlNet_Model_Path}
!mkdir -p {AnimateDiff_Model_Path}

%cd /content/drive/MyDrive/{Workspace}

print('download...')
with capture.capture_output() as cap:
  if Checkpoint_Url:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d ./models/Stable-diffusion -o {Checkpoint_Filename}

  if ControlNet:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_ip2p_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hr16/ControlNet-HandRefiner-pruned/resolve/main/control_sd15_inpaint_depth_hand_fp16.safetensors -d {ControlNet_Model_Path} -o control_sd15_inpaint_depth_hand_fp16.safetensors

  if ControlNet_Extra:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_seg_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_inpaint_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_scribble_fp16.safetensors

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_mlsd_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_normalbae_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_shuffle_fp16.safetensors

  if CotnrolNet_IPAdapter:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter_sd15.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-plus_sd15.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-plus-face_sd15.pth

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-faceid_sd15.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-faceid-plus_sd15.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-faceid-plusv2_sd15.bin

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors -d ./models/Lora -o ip-adapter-faceid-plusv2_sd15_lora.safetensors

  if ControlNet_TemporalNet:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d {ControlNet_Model_Path} -o diff_control_sd15_temporalnet_fp16.safetensors

  if ControlNet_T2I:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_style_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_color_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d {ControlNet_Model_Path} -o t2iadapter_sketch_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d .{ControlNet_Model_Path} -o t2iadapter_zoedepth_sd15v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors -d {ControlNet_Model_Path} -o ioclab_sd15_recolor.safetensors  

  if AnimateDiff:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt -d {AnimateDiff_Model_Path} -o mm_sd_v14.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt -d {AnimateDiff_Model_Path} -o mm_sd_v15.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -d {AnimateDiff_Model_Path} -o mm_sd_v15_v2.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt -d {AnimateDiff_Model_Path} -o v3_sd15_mm.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.ckpt -d {AnimateDiff_Model_Path} -o temporaldiff-v1-animatediff.ckpt
    !ln -s ./extensions/animatediff/model/mm_sd_v14.ckpt {AnimateDiff_Model_Path}/mm_sd_v14.ckpt
    !ln -s ./extensions/animatediff/model/mm_sd_v15.ckpt {AnimateDiff_Model_Path}/mm_sd_v15.ckpt
    !ln -s ./extensions/animatediff/model/mm_sd_v15_v2.ckpt {AnimateDiff_Model_Path}/mm_sd_v15_v2.ckpt
    !ln -s ./extensions/animatediff/model/v3_sd15_mm.ckpt {AnimateDiff_Model_Path}/v3_sd15_mm.ckpt
    !ln -s ./extensions/animatediff/model/temporaldiff-v1-animatediff.ckpt {AnimateDiff_Model_Path}/temporaldiff-v1-animatediff.ckpt

  if AnimateDiff_Extra:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/174464 -d {AnimateDiff_Model_Path} -o improvedHumansMotion_refinedHumanMovement.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/178017 -d {AnimateDiff_Model_Path} -o improved3DMotion_improved3DV1.ckpt
    !ln -s {AnimateDiff_Model_Path}/improvedHumansMotion_refinedHumanMovement.ckpt ./extensions/animatediff/model/improvedHumansMotion_refinedHumanMovement.ckpt
    !ln -s {AnimateDiff_Model_Path}/improved3DMotion_improved3DV1.ckpt ./extensions/animatediff/model/improved3DMotion_improved3DV1.ckpt

!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/direct/{TagVersion}/directui.py -O /content/drive/MyDrive/{Workspace}/directui.py
if TagVersion=='v1.3.2':
  !wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/direct/v1.3.2/modelloader.py -O /content/drive/MyDrive/{Workspace}/modules/modelloader.py
if TagVersion=='v1.10.0':
  !wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/direct/v1.10.0/patches.py -O /content/drive/MyDrive/{Workspace}/modules/patches.py

def clone_pull(git_url, ext_name):
  if os.path.isdir('./extensions/'+ext_name):
    %cd ./extensions/{ext_name}
    !git pull
    %cd ../..
  else:
    !git clone {git_url} ./extensions/{ext_name}

if LCMLora:
  if TagVersion in ["v1.3.2", "v1.4.1", "v1.5.2", "v1.6.0", "v1.7.0"]:
    clone_pull('https://github.com/neuralninja22/lcm-sampler', 'lcm-sampler')
  !wget https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/resolve/main/pytorch_lora_weights.safetensors -O ./models/Lora/lcm-lora-sdv1-5.safetensors

if TemporalKit:
  clone_pull('https://github.com/CiaraStrawberry/TemporalKit', 'TemporalKit')
  !pip install scenedetect

if TemporalNetUtil:
  clone_pull('https://github.com/ninjaneural/temporalnet_util', 'temporalnet_util')
  !pip install ffmpeg-python
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d {ControlNet_Model_Path} -o diff_control_sd15_temporalnet_fp16.safetensors

import sys

sys.argv = [
    sys.argv[0],
    "--xformers",
    "--no-half-vae",
    "--api",
    "--theme",
    "dark",
    '--controlnet-dir',
    ControlNet_Model_Path,
    "--disable-console-progressbars",
]

%cd /content/drive/MyDrive/{Workspace}

print("아래 코드를 실행해주세요!")

In [ ]:
# 직접실행 (검은색 or 흰색 화면만 보이면 스크롤을 올려보세요)
# 실행이 완료되면 url을 클릭해보세요

def run_local_server():
    import IPython
    shell = """
        (async () => {
            const url = new URL(await google.colab.kernel.proxyPort(7860, {'cache': false}));
            link = document.createElement("a");
            link.href = url;
            link.target = 'blank';
            link.innerHTML = url;
            document.body.appendChild(link);
        })();
    """
    script = IPython.display.Javascript(shell)
    IPython.display.display(script)
run_local_server()

from directui import webui
webui()